In [1]:
import os
import random
import gc
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm, trange
import torch
import torch.multiprocessing as mp
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from methods.model import *
from methods.logger import *

from methods.processing import *
from methods.train import *

# 读取数据

In [4]:
mp.set_start_method('spawn', force=True)
main_device_name = 0
print('Read Factor.')
factor = pd.read_pickle('/home/datamake117/data/haris/dataset/total_date.pkl')                      # 日期+股票代码
grouped = pd.read_pickle('/home/datamake117/data/haris/dataset/grouped_adj.pkl').fillna(0)          # 特征
grouped_label = pd.read_pickle('/home/datamake117/data/haris/dataset/grouped_label_adj.pkl')        # 标签
grouped_liquidity = pd.read_pickle('/home/datamake117/data/haris/dataset/grouped_liquidity.pkl')    # 流动性指标
grouped_liquidity.index = grouped_liquidity.index.strftime('%Y%m%d').astype(int)
correlation_df = pd.read_pickle('/home/datamake117/data/haris/dataset/corr_byday_abs.pkl')          # 因子筛选辅助数据
correlation_df.index = correlation_df.index.strftime('%Y%m%d').astype(int)
total_date_list = np.array(factor['date'].drop_duplicates().tolist())                               # 日期列表

Read Factor.


FileNotFoundError: [Errno 2] No such file or directory: '/home/datamake117/data/haris/dataset/total_date.pkl'

# 训练和测试

In [2]:
# 第1轮
print('Round 1.')
round_num = 1
dt1 = int(pd.to_datetime("2020-07-01").strftime('%Y%m%d'))  # 训练集开始时间
dt2 = int(pd.to_datetime("2022-07-01").strftime('%Y%m%d'))  # 验证集开始时间
dt3 = int(pd.to_datetime("2022-12-30").strftime('%Y%m%d'))  # 验证集结束时间
dt4 = int(pd.to_datetime("2023-01-01").strftime('%Y%m%d'))  # 测试集开始时间
dt5 = int(pd.to_datetime("2023-07-01").strftime('%Y%m%d'))  # 测试集结束时间

print(dt1, dt2, dt3, dt4, dt5)

Round 1.
20200701 20220701 20221230 20230101 20230701


In [3]:
# 第2-8轮
total_date_list = np.array(factor['date'].drop_duplicates().tolist())
rolling_step = 3    # 3个月滚动训练
window_size = 24    # 训练集大小
val_size = 3        # 验证集大小
corr_thres = 0.9
for round_num in range(2, 9):
    print('Round %i.' % round_num)
    start_date = pd.to_datetime('2021-01-01')
    dt1 = start_date + relativedelta(months=rolling_step * (round_num - 2)) # 训练集开始时间
    dt2 = dt1 + relativedelta(months=window_size)                           # 验证集开始时间
    dt3 = dt2 + relativedelta(months=val_size)                              # 验证集结束时间
    dt4 = dt3                                                               # 测试集开始时间
    dt5 = dt3 + relativedelta(months=rolling_step)                          # 测试集结束时间
    dt3 = total_date_list[total_date_list < int(dt3.strftime('%Y%m%d'))][-1]
    dt1, dt2, dt3, dt4, dt5 = int(dt1.strftime('%Y%m%d')), int(dt2.strftime('%Y%m%d')), int(dt3), int(dt4.strftime('%Y%m%d')), int(dt5.strftime('%Y%m%d'))
    
    print(dt1, dt2, dt3, dt4, dt5)

NameError: name 'factor' is not defined